<a href="https://colab.research.google.com/github/AlexandreXavierMartins/Sim/blob/main/simpy_aula_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

Sistema Aberto

In [ ]:
import simpy
import numpy as np

# Definição dos parâmetros
tempo_chegada = 30  # Média de chegada dos carros (segundos)
tempo_atendimento = 20  # Média de atendimento (segundos)
tempo_total = 5 * 3600  # Tempo total da simulação (2 horas)
fila = []

def atender_carro(carro_id):
  global fila
  if atendente.count > 0:
    fila.append(carro_id)
    #print('Carros na fila:', fila)
  with atendente.request() as req:
    yield req
    yield env.timeout(np.random.exponential(tempo_atendimento))
    #print(f"Carro {carro_id} atendido em {env.now:.2f}s")
    if len(fila) > 0:
      fila.pop(0)
      #print('Carros na fila:', fila)


# Processo de chegada dos carros
def chegada_carros(env):
    carro_id = 0
    global fila
    while True:
        carro_id += 1
        #print(f"Carro {carro_id} chegou em {env.now:.2f}s")
        #Errado
        """
        if atendente.count > 0:
          fila.append(carro_id)
          print('Carros na fila:', fila)
        with atendente.request() as req:
          yield req
          yield env.timeout(np.random.exponential(tempo_atendimento))
          print(f"Carro {carro_id} atendido em {env.now:.2f}s")
          if len(fila) > 0:
            fila.pop(0)
          print('Carros na fila:', fila)
        #Certo  """
        env.process(atender_carro(carro_id))
        yield env.timeout(np.random.exponential(tempo_chegada))
# Inicialização da simulação
env = simpy.Environment()
atendente = simpy.Resource(env, capacity=1)
env.process(chegada_carros(env))
env.run(until=500)

Coletando Estatísticas:
Compare com as fórmulas da Teoria das Filas Modelo M/M/1

Exercício 1: Crie o processo de atendimento do carro no posto de pedágio. O tempo de atendimento é EXPO(20)
a) Colete a ocupação do atendente
b) O tempo médio de fila
c) O tempo máximo na fila
d) O número médio da fila
e) Verifique o tempo ideal de simulação que mostra que os dados não se alteram muito

In [ ]:
import simpy
import numpy as np

np.random.seed(0)

# Definição dos parâmetros
tempo_chegada = 30  # Média de chegada dos carros (segundos)
tempo_atendimento = 20  # Média de atendimento (segundos)
tempo_total = 480 * 3600  # Tempo total da simulação (2 horas)
fila = []

ocp = 0
tempo_total_fila = 0
cont_fila = 0
t_max = 0
nf = 0
cont = 0
def atender_carro(carro_id):
  global fila, ocp, tempo_total_fila, cont_fila, t_max, nf, cont
  if atendente.count > 0:
    fila.append(carro_id)
    #print('Carros na fila:', fila)
  tempo = env.now #Entra na fila
  nf = nf + len(atendente.queue)
  cont = cont + 1
  with atendente.request() as req:
    yield req
    tempo = env.now - tempo #Sai da fila
    if tempo > t_max:
      t_max = tempo
    tempo_total_fila += tempo
    cont_fila += 1
    #print(f"Carro {carro_id} atendido em {env.now:.2f}s")
    t = np.random.exponential(tempo_atendimento)
    yield env.timeout(t)
    ocp += t
    #print(f"Carro {carro_id} atendido em {env.now:.2f}s")
    if len(fila) > 0:
      fila.pop(0)
      #print('Carros na fila:', fila)


# Processo de chegada dos carros
def chegada_carros(env):
    carro_id = 0
    global fila
    while True:
        carro_id += 1
        #print(f"Carro {carro_id} chegou em {env.now:.2f}s")
        env.process(atender_carro(carro_id))
        yield env.timeout(np.random.exponential(tempo_chegada))
# Inicialização da simulação
env = simpy.Environment()
atendente = simpy.Resource(env, capacity=1)
env.process(chegada_carros(env))
env.run(until=tempo_total)
print("Taxa de ocupação do atendente =", (ocp/tempo_total)*100, '%%')
print(f"Tempo médio dos carros na fila: {tempo_total_fila/cont_fila} segundos")
print(f"Tempo máximo na fila: {t_max} segundos")
print(f"Número médio de carros na fila: {nf/cont}")

Taxa de ocupação do atendente = 66.42844553926894 %%
Tempo médio dos carros na fila: 40.60358357620865 segundos
Tempo máximo na fila: 671.9455228910083 segundos
Número médio de carros na fila: 1.3456781521095356


Incluindo uma parada do recurso

In [36]:
import simpy
import numpy as np

# Definição dos parâmetros
tempo_chegada = 100  # Média de chegada dos carros (segundos)
tempo_atendimento = 20  # Média de atendimento (segundos)
tempo_folga = 5 * 60  # Tempo de folga do atendente (5 minutos)
tempo_total = 3 * 3600  # Tempo total da simulação (2 horas)

fila = []

def gerenciar_folga(env):
  global em_folga, folga_pendente
  while True:
    yield env.timeout(3600)  # Espera 1 hora
    if atendente.count > 0:  # Se estiver atendendo, marca a folga como pendente
      print(f"Atendente adiando folga para terminar atendimento em {env.now:.2f}s")
      folga_pendente = True
    else:
      em_folga = True
      print(f"FOLGA - Atendente iniciando folga em {env.now:.2f}s")
      yield env.timeout(tempo_folga)  # Tempo de descanso
      em_folga = False
      print(f"FOLGA - Atendente retornou ao trabalho em {env.now:.2f}s")

def atender_carro(carro_id):
  global em_folga, folga_pendente, fila
  if atendente.count > 0:
    fila.append(carro_id)
    print('Carros na fila:', fila)
  with atendente.request() as req:
    yield req
    while em_folga:  # Enquanto estiver folgado, espera
      yield env.timeout(1)
    yield env.timeout(np.random.exponential(tempo_atendimento))
    print(f"Carro {carro_id} atendido em {env.now:.2f}s")
    if folga_pendente:  # Se folga estiver pendente, inicia após atendimento
       folga_pendente = False
       print(f"ATD - Atendente iniciando folga em {env.now:.2f}s")
       em_folga = True
       yield env.timeout(tempo_folga)  # Tempo de descanso
       em_folga = False
       print(f"ATD - Atendente retornou ao trabalho em {env.now:.2f}s")
    if len(fila) > 0:
      fila.pop(0)
      print('Carros na fila:', fila)

# Processo de chegada dos carros
def chegada_carros(env):
    carro_id = 0
    while True:
        yield env.timeout(np.random.exponential(tempo_chegada))
        carro_id += 1
        print(f"Carro {carro_id} chegou em {env.now:.2f}s")
        env.process(atender_carro(carro_id))

# Inicialização da simulação
env = simpy.Environment()
atendente = simpy.Resource(env, capacity=1)
folga_pendente = False
em_folga = False
env.process(gerenciar_folga(env))
env.process(chegada_carros(env))
env.run(until=tempo_total)

Carro 1 chegou em 30.86s
Carro 1 atendido em 39.71s
Carro 2 chegou em 69.77s
Carro 2 atendido em 73.71s
Carro 3 chegou em 87.89s
Carro 3 atendido em 140.02s
Carro 4 chegou em 177.48s
Carro 4 atendido em 183.64s
Carro 5 chegou em 190.63s
Carro 5 atendido em 237.52s
Carro 6 chegou em 812.29s
Carro 7 chegou em 827.63s
Carros na fila: [7]
Carro 6 atendido em 836.80s
Carros na fila: []
Carro 7 atendido em 852.93s
Carro 8 chegou em 1090.26s
Carro 8 atendido em 1110.19s
Carro 9 chegou em 1118.64s
Carro 10 chegou em 1131.41s
Carros na fila: [10]
Carro 11 chegou em 1132.99s
Carros na fila: [10, 11]
Carro 9 atendido em 1175.84s
Carros na fila: [11]
Carro 10 atendido em 1181.14s
Carros na fila: []
Carro 11 atendido em 1186.79s
Carro 12 chegou em 1206.93s
Carro 12 atendido em 1248.96s
Carro 13 chegou em 1297.18s
Carro 14 chegou em 1314.51s
Carros na fila: [14]
Carro 13 atendido em 1340.11s
Carros na fila: []
Carro 14 atendido em 1365.27s
Carro 15 chegou em 1413.95s
Carro 15 atendido em 1426.37s
Ca